In [10]:
import os
from pathlib import Path
import sys
import yaml

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [5]:
cwd = os.getcwd()

proj_dir = Path(cwd).parent

os.chdir(proj_dir)

In [6]:
from utils.bathymetry import generate_schematized_bathymetry

In [9]:
load_path = Path("database/transects/")

x0 = np.loadtxt(os.path.join(load_path, 'x0.txt'))
x1 = np.loadtxt(os.path.join(load_path, 'x1.txt'))
x2 = np.loadtxt(os.path.join(load_path, 'x2.txt'))
x3 = np.loadtxt(os.path.join(load_path, 'x3.txt'))

z0 = np.loadtxt(os.path.join(load_path, 'z0.txt'))
z1 = np.loadtxt(os.path.join(load_path, 'z1.txt'))
z2 = np.loadtxt(os.path.join(load_path, 'z2.txt'))
z3 = np.loadtxt(os.path.join(load_path, 'z3.txt'))

xgr = np.loadtxt(os.path.join(load_path, "x_data.grd"))
zgr = np.loadtxt(os.path.join(load_path, "bed_data.dep"))

In [ ]:
def read_config(config_file_path):
    '''
    Creates configuration variables from file
    ------
    config_file: .yaml file
        file containing dictionary with dataset creation information
    ''' 

    class AttrDict(dict):
        """
        This class is used to make it easier to work with dictionaries and allows 
        values to be called similar to attributes
        """
        def __init__(self, *args, **kwargs):
            super(AttrDict, self).__init__(*args, **kwargs)
            self.__dict__ = self
                                
    with open(config_file_path) as f:
        cfg = yaml.safe_load(f)
        
    config = AttrDict(cfg)
            
    for key in cfg:
        config[key] = AttrDict(cfg[key])
            
    return config